Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре).
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

In [ ]:
%pip install matplotlib pandas scikit-learn
%pip install --upgrade keras
%pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0


In [ ]:
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Torch version:", torch.__version__)



Using device: cpu
Torch version: 2.9.0+cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wiki = open('/content/drive/MyDrive/Colab Notebooks/wiki_data.txt').read().split('\n')

In [ ]:
cv = CountVectorizer(max_features=10000)
svd = TruncatedSVD(200)

X = cv.fit_transform(wiki)
X_svd = svd.fit_transform(X)

In [ ]:
embeddings = svd.components_.T

In [ ]:
!pip install --break-system-packages pymorphy3 pymorphy3[fast]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 94.8 MB/s eta 0:00:00


In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()
import re

In [ ]:
def preprocess(text):
    morph_cash = {}
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]

    lemmas = []
    for tok in tokens:
      if tok:
          if tok not in morph_cash:
              lemma = morph.parse(tok)[0].normal_form
              morph_cash.update({tok : lemma})
          lemma = morph_cash[tok]
          lemmas.append(lemma)

    return lemmas

In [ ]:
vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [ ]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
id2word = {i:word for word, i in word2id.items()}

In [ ]:
sentences = []

for text in wiki:
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [ ]:
vocab_size = len(id2word)

In [ ]:
import numpy as np

def pad_sequences(sequences, maxlen, padding='post', value=0):
    """
    sequences: list of lists (разной длины)
    возвращает np.array shape (len(sequences), maxlen)
    """
    res = np.full((len(sequences), maxlen), value, dtype='int64')
    for i, seq in enumerate(sequences):
        if not seq:
            continue
        if len(seq) >= maxlen:
            if padding == 'post':
                res[i] = np.array(seq[:maxlen])
            else:
                res[i] = np.array(seq[-maxlen:])
        else:
            if padding == 'post':
                res[i, :len(seq)] = np.array(seq)
            else:
                res[i, -len(seq):] = np.array(seq)
    return res

In [ ]:
def gen_batches_cbow(sentences, window=5, batch_size=1000):
    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) >= batch_size:
                    X_target_arr = np.array(X_target, dtype='int64')
                    X_context_arr = pad_sequences(X_context, maxlen=window, padding='post', value=0)
                    y_arr = np.array(y, dtype='float32')
                    yield (X_target_arr, X_context_arr), y_arr
                    X_target, X_context, y = [], [], []

In [ ]:
def gen_batches_skipgram(sentences, window=5, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)):
                target = sent[i]
                if target == 0:  # пропускаем PAD
                    continue

                start = max(0, i - window)
                end = min(len(sent), i + window + 1)
                context_words = [sent[j] for j in range(start, end) if j != i and sent[j] != 0]

                for context in context_words:
                    X_target.append(target)
                    X_context.append(context)
                    y.append(1)

                    neg = np.random.randint(1, vocab_size)  # избегаем PAD
                    X_target.append(target)
                    X_context.append(neg)
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target_arr = np.array(X_target, dtype='int64')
                        X_context_arr = np.array(X_context, dtype='int64')
                        y_arr = np.array(y, dtype='float32')
                        yield (X_target_arr, X_context_arr), y_arr
                        X_target, X_context, y = [], [], []


In [ ]:
class CBOWNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim, window_size):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)
        self.window_size = window_size

    def forward(self, target_ids, context_ids):
        """
        target_ids: (batch,)
        context_ids: (batch, window_size)
        """
        t = self.target_emb(target_ids)          # (batch, emb_dim)
        c = self.context_emb(context_ids)        # (batch, window, emb_dim)
        c_sum = c.sum(dim=1)                     # (batch, emb_dim)
        dot = (t * c_sum).sum(dim=1)             # (batch,)
        return dot

In [ ]:
class SkipGramNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim, window_size):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)
        self.window_size = window_size

    def forward(self, target_ids, context_ids):
        t = self.target_emb(target_ids)          # (batch, emb_dim)
        c = self.context_emb(context_ids)        # (batch, emb_dim)
        dot = (t * c).sum(dim=1)                 # (batch,)
        return dot

In [ ]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [ ]:
emb_dim = 100
window_size = 5

model_cb = CBOWNegSampling(vocab_size, emb_dim, window_size).to(device)
model_sg = SkipGramNegSampling(vocab_size, emb_dim, window_size).to(device)

criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer_cb = torch.optim.Adam(model_cb.parameters(), lr=1e-3)
train_cb = gen_batches_cbow(sentences[:19000], window=window_size, batch_size=1000)
valid_cb = gen_batches_cbow(sentences[19000:], window=window_size, batch_size=1000)

In [ ]:
optimizer_sg = torch.optim.Adam(model_sg.parameters(), lr=1e-3)
train_sg = gen_batches_skipgram(sentences[:19000], window=5, batch_size=1000)
valid_sg = gen_batches_skipgram(sentences[19000:], window=5, batch_size=1000)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
steps_per_epoch = 5000
validation_steps = 30
num_epochs = 20

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model_cb.train()
    epoch_loss = 0.0
    for step in range(steps_per_epoch):
        (X_t, X_c), y = next(train_cb)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer_cb.zero_grad()
        logits = model_cb(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer_cb.step()

        epoch_loss += loss.item()
    epoch_loss /= steps_per_epoch

    # validation
    model_cb.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_cb)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = model_cb(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()
    val_loss /= validation_steps

    print(f"Epoch {epoch+1}/{num_epochs} - train loss: {epoch_loss:.4f}, val loss: {val_loss:.4f}")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20 - train loss: 4.8228, val loss: 3.4261
Epoch 2/20 - train loss: 2.9163, val loss: 2.5205
Epoch 3/20 - train loss: 2.2154, val loss: 2.1923
Epoch 4/20 - train loss: 1.7324, val loss: 1.7084
Epoch 5/20 - train loss: 1.4031, val loss: 1.5665
Epoch 6/20 - train loss: 1.1145, val loss: 1.3396
Epoch 7/20 - train loss: 0.9145, val loss: 1.1934
Epoch 8/20 - train loss: 0.7610, val loss: 0.9188
Epoch 9/20 - train loss: 0.6327, val loss: 0.9605
Epoch 10/20 - train loss: 0.5471, val loss: 0.8268
Epoch 11/20 - train loss: 0.4685, val loss: 0.5928
Epoch 12/20 - train loss: 0.4122, val loss: 0.6226
Epoch 13/20 - train loss: 0.3688, val loss: 0.7618
Epoch 14/20 - train loss: 0.3329, val loss: 0.7002
Epoch 15/20 - train loss: 0.3086, val loss: 0.5382
Epoch 16/20 - train loss: 0.2871, val loss: 0.5095
Epoch 17/20 - train loss: 0.2714, val loss: 0.5577
Epoch 18/20 - train loss: 0.2555, val loss: 0.4774
Epoch 19/20 - train loss: 0.2437, val loss: 0.4799
Epoch 20/20 - train loss: 0.2360, val lo

In [ ]:
embeddings_cb = model_cb.context_emb.weight.detach().cpu().numpy()

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model_sg.train()
    epoch_loss = 0.0
    for step in range(steps_per_epoch):
        (X_t, X_c), y = next(train_sg)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer_sg.zero_grad()
        logits = model_sg(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer_sg.step()

        epoch_loss += loss.item()
    epoch_loss /= steps_per_epoch

    # validation
    model_sg.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_sg)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = model_sg(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()
    val_loss /= validation_steps

    print(f"Epoch {epoch+1}/{num_epochs} - train loss: {epoch_loss:.4f}, val loss: {val_loss:.4f}")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20 - train loss: 3.0362, val loss: 2.6354
Epoch 2/20 - train loss: 1.9714, val loss: 1.7347
Epoch 3/20 - train loss: 1.4804, val loss: 1.3780
Epoch 4/20 - train loss: 1.1735, val loss: 0.9078
Epoch 5/20 - train loss: 0.9769, val loss: 0.9572
Epoch 6/20 - train loss: 0.9078, val loss: 0.8242
Epoch 7/20 - train loss: 0.7376, val loss: 0.7419
Epoch 8/20 - train loss: 0.7303, val loss: 0.7070
Epoch 9/20 - train loss: 0.6868, val loss: 0.6745
Epoch 10/20 - train loss: 0.6367, val loss: 0.6340
Epoch 11/20 - train loss: 0.6111, val loss: 0.5328
Epoch 12/20 - train loss: 0.5725, val loss: 0.6067
Epoch 13/20 - train loss: 0.5548, val loss: 0.5248
Epoch 14/20 - train loss: 0.5357, val loss: 0.5866
Epoch 15/20 - train loss: 0.5214, val loss: 0.5092
Epoch 16/20 - train loss: 0.4988, val loss: 0.5034
Epoch 17/20 - train loss: 0.4854, val loss: 0.5196
Epoch 18/20 - train loss: 0.4763, val loss: 0.4706
Epoch 19/20 - train loss: 0.4606, val loss: 0.4641
Epoch 20/20 - train loss: 0.4638, val lo

In [ ]:
embeddings_sg = model_sg.target_emb.weight.detach().cpu().numpy()

In [ ]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [ ]:
most_similar('техника', embeddings_cb)

['техника',
 'задача',
 'работа',
 'проектирование',
 'машина',
 'графический',
 'способ',
 'стандартный',
 'материал',
 'производство']

['техника',
 'задача',
 'работа',
 'проектирование',
 'машина',
 'графический',
 'способ',
 'стандартный',
 'материал',
 'производство']

In [ ]:
most_similar('спорт', embeddings_cb)

['спорт',
 'гастролировать',
 'футбол',
 'соревнование',
 'сборная',
 'спортсмен',
 'атлетика',
 'олимпийский',
 'олимпиада',
 'любительский']

['спорт',
 'гастролировать',
 'футбол',
 'соревнование',
 'сборная',
 'спортсмен',
 'атлетика',
 'олимпийский',
 'олимпиада',
 'любительский']

In [ ]:
most_similar('город', embeddings_cb)

['город',
 'юг',
 'территория',
 'крепость',
 'регион',
 'ныне',
 'администрация',
 'житель',
 'столица',
 'улица']

['город',
 'юг',
 'территория',
 'крепость',
 'регион',
 'ныне',
 'администрация',
 'житель',
 'столица',
 'улица']

In [ ]:
most_similar('техника', embeddings_sg)

['техника',
 'разработать',
 'технический',
 'задача',
 'автомобиль',
 'модель',
 'определять',
 'обеспечение',
 'порядок',
 'использоваться']

['техника',
 'разработать',
 'технический',
 'задача',
 'автомобиль',
 'модель',
 'определять',
 'обеспечение',
 'порядок',
 'использоваться']

In [ ]:
most_similar('спорт', embeddings_sg)

['спорт',
 'соревнование',
 'сборная',
 'чемпион',
 'тренер',
 'медаль',
 'чемпионат',
 'проводиться',
 'выступать',
 '1972']

['спорт',
 'соревнование',
 'сборная',
 'чемпион',
 'тренер',
 'медаль',
 'чемпионат',
 'проводиться',
 'выступать',
 '1972']

In [ ]:
most_similar('город', embeddings_sg)

['город',
 'находиться',
 'территория',
 'западный',
 'восточный',
 'граница',
 'пункт',
 'провинция',
 'центральный',
 'деревня']

['город',
 'находиться',
 'территория',
 'западный',
 'восточный',
 'граница',
 'пункт',
 'провинция',
 'центральный',
 'деревня']


# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 26.8 MB/s eta 0:00:00


In [ ]:
import gensim

In [ ]:
texts = [preprocess(text) for text in wiki]

In [ ]:
w2v_sg = gensim.models.Word2Vec(texts,
                             vector_size=200,
                             min_count=40,
                             max_vocab_size=12000,
                             window=7,
                             epochs=10,
                             sg=1,
                             hs=0,
                             negative=7,
                             sample=1e-4,
                             ns_exponent=0.7,
                             cbow_mean=1
                            )

In [ ]:
ft = gensim.models.FastText(texts, min_n=4, max_n=9)

In [ ]:
w2v_sg.wv.most_similar('спорт')

[('атлетика', 0.6338723301887512),
 ('соревнование', 0.6100934147834778),
 ('лыжный', 0.5939009785652161),
 ('спортивный', 0.5727084279060364),
 ('дзюдо', 0.5376569032669067),
 ('спортсмен', 0.5372270345687866),
 ('сборная', 0.514384925365448),
 ('тренер', 0.5093189477920532),
 ('заслужить', 0.5084306001663208),
 ('футбол', 0.5028868913650513)]

[('атлетика', 0.6338723301887512),
 ('соревнование', 0.6100934147834778),
 ('лыжный', 0.5939009785652161),
 ('спортивный', 0.5727084279060364),
 ('дзюдо', 0.5376569032669067),
 ('спортсмен', 0.5372270345687866),
 ('сборная', 0.514384925365448),
 ('тренер', 0.5093189477920532),
 ('заслужить', 0.5084306001663208),
 ('футбол', 0.5028868913650513)]

In [ ]:
w2v_sg.wv.most_similar('город')

[('столица', 0.6003149151802063),
 ('центр', 0.5858110189437866),
 ('посёлок', 0.5526561141014099),
 ('район', 0.53823322057724),
 ('городской', 0.5079519748687744),
 ('населить', 0.4947884976863861),
 ('административный', 0.48509329557418823),
 ('округ', 0.4850730001926422),
 ('территория', 0.48490747809410095),
 ('пункт', 0.47909989953041077)]

[('столица', 0.6003149151802063),
 ('центр', 0.5858110189437866),
 ('посёлок', 0.5526561141014099),
 ('район', 0.53823322057724),
 ('городской', 0.5079519748687744),
 ('населить', 0.4947884976863861),
 ('административный', 0.48509329557418823),
 ('округ', 0.4850730001926422),
 ('территория', 0.48490747809410095),
 ('пункт', 0.47909989953041077)]

In [ ]:
ft.wv.most_similar('спорт')

[('спортзал', 0.9811062216758728),
 ('спорт»', 0.9807443618774414),
 ('спорта»', 0.9767930507659912),
 ('«спорт', 0.9590727090835571),
 ('спорткара', 0.9563000202178955),
 ('велоспорт', 0.9469243288040161),
 ('спортинг»', 0.9319948554039001),
 ('спортивная»', 0.9056984186172485),
 ('паспорт', 0.8984086513519287),
 ('2:3', 0.8909982442855835)]

[('спортзал', 0.9811062216758728),
 ('спорт»', 0.9807443618774414),
 ('спорта»', 0.9767930507659912),
 ('«спорт', 0.9590727090835571),
 ('спорткара', 0.9563000202178955),
 ('велоспорт', 0.9469243288040161),
 ('спортинг»', 0.9319948554039001),
 ('спортивная»', 0.9056984186172485),
 ('паспорт', 0.8984086513519287),
 ('2:3', 0.8909982442855835)]

In [ ]:
ft.wv.most_similar('город')

[('ужгород', 0.9776901006698608),
 ('«город', 0.975427508354187),
 ('город»', 0.9745731353759766),
 ('городе»', 0.9728744626045227),
 ('городец', 0.9709781408309937),
 ('горо', 0.964139997959137),
 ('огород', 0.9580005407333374),
 ('городня', 0.9549098610877991),
 ('горох', 0.9506648778915405),
 ('городов', 0.9490054845809937)]

[('ужгород', 0.9776901006698608),
 ('«город', 0.975427508354187),
 ('город»', 0.9745731353759766),
 ('городе»', 0.9728744626045227),
 ('городец', 0.9709781408309937),
 ('горо', 0.964139997959137),
 ('огород', 0.9580005407333374),
 ('городня', 0.9549098610877991),
 ('горох', 0.9506648778915405),
 ('городов', 0.9490054845809937)]

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор.
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/labeled.csv')

In [ ]:
data['norm_text'] = data.comment.apply(preprocess)

In [ ]:
import numpy as np

def tokens_to_avg_emb(tokens, model):
    vecs = []
    for token in tokens:
        if token in model.wv:
            vecs.append(model.wv[token])
    if not vecs:
        return np.zeros(model.wv.vector_size)
    return np.mean(vecs, axis=0)

X_emb = np.array([tokens_to_avg_emb(tokens, w2v_sg) for tokens in data['norm_text']])

In [ ]:
y = data['toxic'].values
X_train, X_test, y_train, y_test = train_test_split(
    X_emb, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.4f}")

F1-score: 0.6308


F1-score: 0.6308

# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

In [ ]:
#ЗАДАНИЕ СДЕЛАНО В ОТДЕЛЬНОЙ ТЕТРАДКЕ